#### Note:

Make sure that you have these two scripts running at the same time while running this

ark regsitry start

python sim_node.py

In [7]:
import numpy as np
from ark.client.comm_infrastructure.instance_node import InstanceNode
from arktypes import joint_group_command_t, task_space_command_t, joint_state_t
from arktypes.utils import unpack, pack

SIM = True


class FrankaControllerNode(InstanceNode):

    def __init__(self):
        '''
        Initialize the FrankaJointController.
        This class is responsible for controlling the Franka robot's joints.
        '''
        super().__init__("FrankaJointController")

        if SIM == True:
            self.joint_group_command = self.create_publisher("Franka/joint_group_command/sim", joint_group_command_t)
            self.task_space_command = self.create_publisher("Franka/cartesian_command/sim", task_space_command_t)

            self.state = self.create_listener("Franka/joint_states/sim", joint_state_t)


controller = FrankaControllerNode()

[WARNING] [11:54:17.060568] - No global configuration provided. Using default system configuration.
[INFO] [11:54:17.109674] - Service: Successfully registered '__DEFAULT_SERVICE/GetInfo/FrankaJointController_1a0bcfa2-a34f-4e5e-b403-e6868c9fe744' with registry.
[INFO] [11:54:17.139916] - Service: Successfully registered '__DEFAULT_SERVICE/SuspendNode/FrankaJointController_1a0bcfa2-a34f-4e5e-b403-e6868c9fe744' with registry.
[INFO] [11:54:17.163540] - Service: Successfully registered '__DEFAULT_SERVICE/RestartNode/FrankaJointController_1a0bcfa2-a34f-4e5e-b403-e6868c9fe744' with registry.
[ERROR] [11:54:17.173760] - Couldn't load config for other 'FrankaJointController'
[OK] [11:54:17.177048] - setup publisher Franka/joint_group_command/sim[joint_group_command_t]
[OK] [11:54:17.179968] - setup publisher Franka/cartesian_command/sim[task_space_command_t]
[OK] [11:54:17.181656] - subscribed to Franka/joint_states/sim[joint_state_t]


None


---

# Joint Group Control

In [8]:
joint_command = [-0.3, 0.1, 0.3, -1.4, 0.1, 1.8, 0.6, 0]
# Position Control
controller.joint_group_command.publish(pack.joint_group_command(joint_command, "all"))

In [10]:
gripper_command = [-1]  # -1 close, 1 open
# Velocity Control 
controller.joint_group_command.publish(pack.joint_group_command(gripper_command, "gripper"))

In [11]:
arm_command = [-0.3, 0.5, 0.3, -1.0, 0.1, 1.8, 0.1]
# Position Control 
controller.joint_group_command.publish(pack.joint_group_command(arm_command, "arm"))

---

# Task Space Control

In [30]:
xyz_command = np.array([0.3, 0.4, 0.8])
quaternion_command = np.array([1, 0.0, 0.0, 0.0])  # xyz-w
gripper = 1.0  # 0.0 close, 1.0 open

controller.task_space_command.publish(
    pack.task_space_command("all", xyz_command, quaternion_command, gripper))

---


# Joint States

In [26]:
unpack.joint_state(controller.state.get())

({'seq': 0, 'stamp': {'sec': 0, 'nsec': 0}, 'frame_id': ''},
 ['panda_joint1',
  'panda_joint2',
  'panda_joint3',
  'panda_joint4',
  'panda_joint5',
  'panda_joint6',
  'panda_joint7',
  'panda_finger_joint1',
  'panda_finger_joint2'],
 array([ 3.32046132e-01, -2.17307877e-01,  5.52577220e-01, -2.11408249e+00,
         1.20972593e-01,  1.92591533e+00,  8.31678377e-01, -1.00911550e-21,
         3.80750550e-08]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0.]))